<a href="https://colab.research.google.com/github/suhaodatascichem/NIR-calibration---NN/blob/main/NIR%20calibration%20-%20NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from setuptools import setup

setup(
    name="brukeropusreader",
    use_scm_version=True,
    description="Bruker OPUS File Reader",
    author="QED",
    author_email="brukeropusreader-dev@qed.ai",
    packages=["brukeropusreader"],
    # install_requires=["numpy>=1.13.3", "scipy>=0.19.1"],
    license="LGPLv3+",
    url="https://github.com/qedsoftware/brukeropusreader",
)

/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/dist.py:261: UserWarning: Unknown distribution option: 'use_scm_version'
  warnings.warn(msg)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/fancy_getopt.py", line 245, in getopt
    opts, args = getopt.getopt(args, short_opts, self.long_opts)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/getopt.py", line 95, in getopt
    opts, args = do_shorts(opts, args[0][1:], shortopts, args[1:])
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/getopt.py", line 195, in do_shorts
    if short_has_arg(opt, shortopts):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/getopt.py", line 211, in short_has_arg
    raise GetoptError(_('option -%s not recognized') % opt, opt)
getopt.GetoptError: option -f not recognized

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/core.py", line 170, in setup
    ok = dist.p

TypeError: object of type 'NoneType' has no len()

In [3]:
# read bruker files - brukeropusreader
!pip install brukeropusreader

  Preparing metadata (setup.py) ... done
  Created wheel for brukeropusreader: filename=brukeropusreader-1.3.4-py3-none-any.whl size=5303 sha256=215b373e8a98f62ea75ff9fee8f76c2ec2b1d91e721577b24f1b354976abf6c6
  Stored in directory: /root/.cache/pip/wheels/33/b6/88/6f85a1072c5d56dc11f6f28858fc7049281172fa48a5100908
Successfully built brukeropusreader


In [ ]:
import argparse
import sys

from brukeropusreader import read_file


def main(path_to_file):
    print(f"Reading opus file from path" f"{path_to_file}")
    opus_data = read_file(path_to_file)

    print(f"Data fields: " f"{list(opus_data.keys())}")

    ab_x = opus_data.get_range("AB")
    # the "AB" data can contain more null values at the end (at least 1)
    # so the getting useful data requires slicing the array:
    abs = opus_data["AB"][0:len(ab_x)]
    print(f"Absorption spectrum range: " f"{ab_x[0]} {ab_x[-1]}")
    print(f"Absorption elements num: " f'{len(abs)}')

    try:
        import matplotlib.pyplot as plt

        print("Plotting AB")
        plt.plot(opus_data.get_range("AB"), abs)
        plt.show()

        print("Plotting interpolated AB")
        plt.plot(*opus_data.interpolate(ab_x[0], ab_x[-1], 100))
        plt.show()

    except ImportError:
        print(f"Install matplotlib to plot spectra")


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("opus_path", help="Path to opus file", action="store")
    args = parser.parse_args()
    main(sys.argv[1])

In [ ]:
from brukeropusreader import read_file

opus_data = read_file('opus_file.0')

print(f'Parsed fields: '
      f'{list(opus_data.keys())}')

print(f'Absorption spectrum: '
      f'{opus_data["AB"]}'